In [3]:

def calc_x_reserves(APY,y_reserves,days_until_maturity,time_stretch):
    t=days_until_maturity/(365*time_stretch)
    T=days_until_maturity/365
    return y_reserves*(-(2/((1-T*APY/100)**(1/t)-1))-2)

def calc_liquidity(target_liquidity, market_price, apy, days_until_maturity, time_stretch):
  y_reserves = target_liquidity/market_price
  t=days_until_maturity/(365*time_stretch)
  liquidity = 0

  # dirty hack to ensure that we dont overshoot the target liquidity
  while True:
      x_reserves = calc_x_reserves(apy,y_reserves,days_until_maturity,time_stretch)
      total_supply=x_reserves + y_reserves
      liquidity=x_reserves*market_price+y_reserves*market_price*calc_spot_price(x_reserves,y_reserves,total_supply,t)
      if liquidity > target_liquidity:
          y_reserves-=100
      else:
          break
  return (x_reserves,y_reserves)

def calc_time_stretch(apy):
  return 3.09396 /( 0.02789 * apy)

def calc_spot_price(x_reserves,y_reserves,total_supply,t):
    return 1/pow((y_reserves+total_supply)/x_reserves,t)

def calc_apy(price,days_until_maturity):
  T=days_until_maturity/365
  return (1-price)/T * 100

def calc_out_given_in(in_,in_reserves,out_reserves,token_out,g,t):
    k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
    without_fee = out_reserves - pow(k-pow(in_reserves+in_,1-t),1/(1-t))
    if token_out == "base":
        fee =  (in_-without_fee)*g
        with_fee = without_fee-fee
    elif token_out == "pt":
        fee =  (without_fee-in_)*g
        with_fee = without_fee-fee
    return (with_fee,without_fee,fee)


In [4]:
###############################
# USER INPUTS
target_liquidty = 50000000  # total pool liquidity (USD)
trade_value = 500000 # (USD)
APY = 10 # Fixed Rate APY of PT
days_until_maturity = 90
#############################

market_price = 1 # 1 DAI = 1 USD
time_stretch = calc_time_stretch(APY)
t=days_until_maturity/(365*time_stretch)
amount=trade_value/market_price
fee=0.1
(x_reserves, y_reserves) = calc_liquidity(target_liquidty, market_price, APY, days_until_maturity, time_stretch)
print("initial reserves: {:,.2f} DAI & {:,.2f} PT".format(x_reserves,y_reserves))
total_supply=x_reserves+y_reserves
spot_price=1/calc_spot_price(x_reserves,y_reserves,total_supply,t)
print("spot price: {:}".format(spot_price))
print("initial apy: {:,.2f}".format(calc_apy(1/spot_price,days_until_maturity)))
(with_fee,without_fee,fee)=calc_out_given_in(amount,x_reserves,y_reserves+total_supply,"pt",fee,t)
without_slippage_or_fee = spot_price * amount
slippage = abs(without_slippage_or_fee-without_fee)
print("\nTrade: {:,.2f} DAI".format(amount))
print("Output without fees and without slippage: {:,.2f} PTs".format(without_slippage_or_fee))
print("Slippage: {:,.2f} PTs".format(slippage))
print("Total fee: {:,.2f} PTs".format(fee))
print("Actual Output: {:,.2f} PTs".format(with_fee))
print("Resulting APY: " + str(calc_apy(amount/with_fee,days_until_maturity)))

initial reserves: 24,852,922.41 DAI & 25,782,800.00 PT
spot price: 1.0252808988764046
initial apy: 10.00

Trade: 500,000.00 DAI
Output without fees and without slippage: 512,640.45 PTs
Slippage: 152.13 PTs
Total fee: 1,248.83 PTs
Actual Output: 511,239.49 PTs
Resulting APY: 8.916052490496506
